### Microoptimisation

Write a crossmatch function for two catalogues to within a maximum radius. The catalogues are 2D NumPy arrays of RA and declination in degrees.

Your function should convert all the coordinates to radians before it starts crossmatching. It should return 3 values:

A list of tuples of matched IDs and their distance in degrees;
A list of unmatched IDs from the first catalogue;
The time taken (in seconds) to run the crossmatcher.
Both catalogues are given as an N×2 NumPy array of floats. Each row contains the coordinates of a single object. The two columns are the RA and declination.

An object's ID is the index of its row, starting at 0. Your function should work with input catalogues with any number of objects.

Here's an example of how your function should work:

>>> cat1 = np.array([[180, 30], [45, 10], [300, -45]])
>>> cat2 = np.array([[180, 32], [55, 10], [302, -44]])
>>> matches, no_matches, time = crossmatch(cat1, cat2, 5)
>>> print('matches:', matches)
matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
>>> print('unmatched:', no_matches)
unmatched: [1]
>>> print('time taken:', time_taken)
time taken: 0.00022228599118534476

In [1]:
import numpy as np
import time

def angular_dist(r1, d1, r2, d2):
  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
  start = time.perf_counter()
  
  max_radius = np.radians(max_radius)
  
  matches = []
  no_matches = []

  # Convert coordinates to radians
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)

  for id1, (ra1, dec1) in enumerate(cat1):
    min_dist = np.inf
    min_id2 = None
    for id2, (ra2, dec2) in enumerate(cat2):
      dist = angular_dist(ra1, dec1, ra2, dec2)
      if dist < min_dist:
        min_id2 = id2
        min_dist = dist
        
    # Ignore match if it's outside the maximum radius
    if min_dist > max_radius:
      no_matches.append(id1)
    else:
      matches.append((id1, min_id2, np.degrees(min_dist)))
    
  time_taken = time.perf_counter() - start
  return matches, no_matches, time_taken

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.0008487999999999829
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
time taken: 0.005149900000006369


### Vectorisation

Copy and modify your previous angular_dist and crossmatch in radians functions to calculate the distances to all of the objects in the second catalogue using NumPy arrays.

The return values should behave the same way as the original function, given the same arguments, except time_taken should be noticeably smaller for large catalogues.

For example, angular_dist should work with an array second catalogue:

>>> ra1, dec1 = np.radians([180, 30])
>>> cat2 = [[180, 32], [55, 10], [302, -44]]
>>> cat2 = np.radians(cat2)
>>> ra2s, dec2s = cat2[:,0], cat2[:,1]
>>> dists = angular_dist(ra1, dec1, ra2s, dec2s)
>>> print(np.degrees(dists))
array([2,113.72587199,132.64478705])

In [3]:
# Write your crossmatch function here.
import numpy as np
import time

def angular_dist(r1,d1,r2,d2):
  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
  start = time.perf_counter()
  matches = []
  no_matches = []
  max_radius = np.radians(max_radius)
  
  #converting values to radians
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)

  ra2s, dec2s = cat2[:, 0], cat2[:, 1]
  
  for id1, (ra1, dec1) in enumerate(cat1):
    dists = angular_dist(ra1, dec1, ra2s, dec2s)
    min_id2 = np.argmin(dists)
    min_dist = dists[min_id2]
    
    if min_dist > max_radius:
      no_matches.append(id1)
    else:
      matches.append((id1, min_id2, np.degrees(min_dist)))

  time_taken = time.perf_counter() - start
  return matches, no_matches, time_taken

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  ra1, dec1 = np.radians([180, 30])
  cat2 = [[180, 32], [55, 10], [302, -44]]
  cat2 = np.radians(cat2)
  ra2s, dec2s = cat2[:,0], cat2[:,1]
  dists = angular_dist(ra1, dec1, ra2s, dec2s)
  print(np.degrees(dists))

  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)


[  2.         113.72587199 132.64478705]
matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.0002261000001908542
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
time taken: 0.0004426999998941028


### Break Out

Copy your crossmatch solution from Microoptimisation and modify it so that it sorts catalogue 2 by declination and breaks out of the inner loop early.

Your crossmatch should break out of the loop over the second catalogue when the declination reaches dec1 + max_radius.

The return values should behave the same way as the original function, given the same arguments, except time_taken should be noticeably smaller for large catalogues.

We will test your function on random input arrays. We've included the function create_cat in the starting file to generate random arrays so you can test your function yourself.

Hint
The NumPy function np.argsort may be useful to you. It returns a list of sorted indices so that:

sort_ind = np.argsort(decs)

dec_sorted = decs[sort_ind]

gives the same result as dec_sorted = np.sort(decs). So if you find row n in decs_sorted is a match, then that means row sort_ind[n] is your match in the original decs array.

In [5]:
# Write your crossmatch function here.
import numpy as np
import time

def angular_dist(r1,d1,r2,d2):
  a = np.sin(np.abs(d1 - d2)/2)**2
  b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
  return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
  start = time.perf_counter()
  matches = []
  no_matches = []
  max_radius = np.radians(max_radius)
  
  #converting values to radians
  cat1 = np.radians(cat1)
  cat2 = np.radians(cat2)

  order = np.argsort(cat2[:, 1])
  cat2_ordered = cat2[order]
  
  for id1, (ra1, dec1) in enumerate(cat1):
    min_dist = np.inf
    min_id2 = None
    max_dec = dec1 + max_radius
    for id2, (ra2, dec2) in enumerate(cat2_ordered):
      if dec2 > max_dec:
        break
      dist = angular_dist(ra1, dec1, ra2, dec2)
      if dist < min_dist:
        min_dist = dist
        min_id2 = order[id2]
    
    # Ignore match if it's outside the maximum radius
    if min_dist > max_radius:
      no_matches.append(id1)
    else:
      matches.append((id1, min_id2, np.degrees(min_dist)))

  time_taken = time.perf_counter() - start
  return matches, no_matches, time_taken

# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)


matches: [(0, 0, 2.0000000000000027), (2, 2, 1.7420109046547023)]
unmatched: [1]
time taken: 0.005155900000090696
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
time taken: 0.004903999999896769


Copy crossmatch from Break out and modify it to only loop through objects in the second catalogue with declinations between dec1 - max_radius and dec1 + max_radius using binary search.

Your crossmatch should use np.searchsorted to find the starting point (just before dec1 - max_radius) and then break out of the loop when the declination reaches dec1 + max_radius.

The return values should behave the same way as the original function, given the same arguments, except time_taken should be noticeably smaller for large catalogues.

In [37]:
import numpy as np
from time import time

def angular_dist_rad(r1, d1, r2, d2):
    angle = 2*np.arcsin(np.sqrt(np.sin(abs(d1 - d2)/2)**2 
                        + np.cos(d1)*np.cos(d2)*np.sin(abs(r1 - r2)/2)**2))
    return angle

def crossmatch(cat1, cat2 , max_radius):
    start_time = time()
    max_radius = np.radians(max_radius)
    matches = []
    no_matches = []
    
    # Convert coordinates to radians
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)
    
    # Find ascending declination order of second catalogue
    asc_dec = np.argsort(cat2[:, 1])
    cat2_sorted = cat2[asc_dec]
    dec2_sorted = cat2_sorted[:, 1]
    
    for id1, (ra1, dec1) in enumerate(cat1):
        closest_dist = np.inf
        closest_id2 = None
        
        # Declination search box
        min_dec = dec1 - max_radius
        max_dec = dec1 + max_radius
        
        # Start and end indices of the search
        start = dec2_sorted.searchsorted(min_dec, side='left')
        end = dec2_sorted.searchsorted(max_dec, side='right')
        
        for s_id2, (ra2, dec2) in enumerate(cat2_sorted[start:end+1], start):
            dist = angular_dist_rad(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_sorted_id2 = s_id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            closest_id2 = asc_dec[closest_sorted_id2]
            matches.append([id1, closest_id2, np.degrees(closest_dist)])
    
    time_taken = time() - start_time
    return matches, no_matches, time_taken
  
  # You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2 , 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)


matches: [[0, 0, 2.0000000000000027], [2, 2, 1.7420109046547023]]
unmatched: [1]
time taken: 0.008385419845581055
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
time taken: 0.0


### k-d Tree

Copy your crossmatch solution from Microoptimisation and (substantially!) modify it to use Astropy to perform the matching.

The return values should behave the same way as the original function, given the same arguments, except time_taken should be noticeably smaller for large catalogues.

In [42]:
# Write your crossmatch function here.
import numpy as np
import time
from astropy.coordinates import SkyCoord
from astropy import units as u

def crossmatch(cat1, cat2, max_radius):
  start = time.perf_counter()
  matches = []
  no_matches = []
  
  # Convert coordinates to radians
  sky_cat1 = SkyCoord(cat1*u.degree, frame = 'icrs')
  sky_cat2 = SkyCoord(cat2*u.degree, frame = 'icrs')

  #performing crossmatching
  closest_ids, closest_dists, closest_dists3d = sky_cat1.match_to_catalog_sky(sky_cat2)
  
  for id1, (closest_id2, dist) in enumerate(zip(closest_ids, closest_dists)):
    closest_dist = dist.value
        
    # Ignore match if it's outside the maximum radius
    if closest_dist > max_radius:
      no_matches.append(id1)
    else:
      matches.append([id1, closest_id2, closest_dist])
    
  time_taken = time.perf_counter() - start
  return matches, no_matches, time_taken


# You can use this to test your function.
# Any code inside this `if` statement will be ignored by the automarker.
if __name__ == '__main__':
  # The example in the question
  cat1 = np.array([[180, 30], [45, 10], [300, -45]])
  cat2 = np.array([[180, 32], [55, 10], [302, -44]])
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)

  # A function to create a random catalogue of size n
  def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))

  # Test your function on random inputs
  np.random.seed(0)
  cat1 = create_cat(10)
  cat2 = create_cat(20)
  matches, no_matches, time_taken = crossmatch(cat1, cat2, 5)
  print('matches:', matches)
  print('unmatched:', no_matches)
  print('time taken:', time_taken)


matches: [[0, 0, 2.0000000000000036], [2, 2, 1.7420109046547163]]
unmatched: [1]
time taken: 0.007374400000117021
matches: []
unmatched: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
time taken: 0.00748829999974987
